In [14]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from timeit import default_timer as timer
import cv2

In [13]:
rectSize = 5
inputImagePath = "PostDisaster"
inputImageFile = "channelviewPost.tif"
#inputImage = Image.open(inputImagePath + '/' + inputImageFile)
#inputImageAlpha.load() # required for png.split()
#inputImage = Image.new("RGB", inputImageAlpha.size, (255, 255, 255))
#inputImage.paste(inputImageAlpha, mask=inputImageAlpha.split()[3])
#inputImage = inputImage.resize((512,512),Image.ANTIALIAS)
#x = inputImage.load()
#print(x[0,0])
inputImage = cv2.imread(inputImagePath + '/' + inputImageFile)
print(inputImage.shape)
inputImage = cv2.resize(inputImage,(1512,1512))
ipx = inputImage.shape[0]
ipy = inputImage.shape[1]
#inputImage[100,100] = [0,0,0]
#print(inputImage)
# for x in range(9351):
#     for y in range(9351):
#         inputImage[x,y] = [0,0,0]
#         print(inputImage[x,y])
#ipx, ipy = inputImage.size
#print(ipx,ipy)
outputImagePath = "RoadSegments"
outputImageFile = "channelviewPostRoad9.png"
outputImage = Image.new('RGB', (ipx,ipy))
opx, opy = outputImage.size
#ipx, ipy, opx, opy = 100, 100, 100, 100




(9351, 9351, 3)


In [ ]:
img = cv

In [15]:
numeric_columns = []
for i in range(1,76):
    numeric_columns.append("c"+str(i))
#print(numeric_columns)
numeric_features = [ tf.feature_column.numeric_column(key = column)
                    for column in numeric_columns]
#print(numeric_features[0])
trainingSteps = 1000
total = 5000
classifierConfig = tf.estimator.RunConfig(save_checkpoints_secs = None, save_checkpoints_steps = trainingSteps)
hiddenUnits = [100, 150, 100, 50]
classes = 2
modelDir = 'Model3'

classifier = tf.estimator.DNNClassifier(feature_columns = numeric_features,
                                             hidden_units = hiddenUnits,
                                             n_classes = classes,
                                             model_dir = modelDir,
                                             config = classifierConfig)

INFO:tensorflow:Using config: {'_model_dir': 'Model3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000256DA9122E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
def extractFeatures(y):
    features = np.zeros((( (ipy - ((rectSize//2)*2))), rectSize*rectSize*3), dtype=np.int)
    print(features.shape)
    rowIndex = 0
    
    for x in range(rectSize//2, ipy -(rectSize//2) ):
        rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1,y + (rectSize//2) + 1 )
        subImage = inputImage[rect[0]:rect[2], rect[1]:rect[3]]
        colIndex = 0       
        for j in range(rectSize):
            for i in range(rectSize):
                features[rowIndex, colIndex] = subImage[i, j][2]
                colIndex += 1
                features[rowIndex, colIndex] = subImage[i, j][1]
                colIndex += 1
                features[rowIndex, colIndex] = subImage[i, j][0]
                colIndex += 1
        rowIndex += 1
        
    x_predict = pd.DataFrame(data = features, columns = col, index = None)
    #print(x_predict)
    predict_input_fn = tf.estimator.inputs.pandas_input_fn(
                        x = x_predict,
                        y = None,
                        batch_size = 32,
                        shuffle = False,
                        num_epochs = 1    
                        )
    predictions = list(classifier.predict(input_fn = predict_input_fn))
    #print(len(predictions))
    #print(opx*opy)
    rowIndex = 0

    for x in range(rectSize//2, opy -(rectSize//2)):
        if(rowIndex >= len(predictions)):
            outputImgPixels[x, y] = (0, 0, 0)
            continue;
        if(predictions[rowIndex]['class_ids'][0]):
            outputImgPixels[x, y] = (255, 255, 255)
        else:
            outputImgPixels[x, y] = (0, 0, 0)
        rowIndex += 1

In [17]:
start = timer()

col = []
for i in range(1,76):
    col.append("c"+str(i))
print(col)
outputImgPixels = outputImage.load()
for x in range(rectSize//2, ipx -(rectSize//2) ):
    extractFeatures(x)
    print(x)
outputImage.save(outputImagePath + '/' + outputImageFile, 'PNG')

timeTaken = timer()-start
print(timeTaken)

['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 'c65', 'c66', 'c67', 'c68', 'c69', 'c70', 'c71', 'c72', 'c73', 'c74', 'c75']
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running lo

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
30
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
31
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
32
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
60
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
61
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
62
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
63
(1508, 75)
INFO:tensorflow:Callin

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
90
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
91
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
92
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
93
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

INFO:tensorflow:Done running local_init_op.
120
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
121
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
122
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
123
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorf

...

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1385
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1386
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1387
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1388
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1415
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1416
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1417
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1418
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1445
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1446
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1447
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1448
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1475
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1476
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1477
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1478
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1505
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1506
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1507
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Model3\model.ckpt-9000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1508
(1508, 75)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:t